# Comment

Датасет http://www.cs.cmu.edu/~enron/
Уже скаченный и лежит в папке '/mnt/data/molchanov/datasets/maildir'

! pip install mail-parser

StreamingGraph -  словарь графов (1 граф соответствует 1 дню) ключ - дата (день)
Каждый граф - объект типа Graph, который хранит вершины и ребра (Edge) в виде списка смежности, 
что представляет из себя словарь словарей, ключи которых - имена нод графов в данном случае емэил адреса отправителя 
и получающего, например Graph['sender1@mail.ru']['reciver2@mail.ru'] это ребро, объект класса Edge - распарсенные письма
между 'sender1@mail.ru' и 'reciver2@mail.ru' в течении дня. Edge - Хранит в себе словарь свойств, на данный момент там есть число 
писем и Subject письма.

In [ ]:
import time
import os
from os import listdir
from os import path as osp
import tqdm
import re
from collections import defaultdict
import numpy as np
from copy import copy, deepcopy
import datetime
import mailparser
from multiprocessing import Pool
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from math import ceil

base_path = '/mnt/data/molchanov/datasets/maildir'
MIN_YEAR = 1999
MIN_DATE = datetime.datetime(1999, 7, 1)
MAIL2NICK = dict()
NICK2MAIL = dict()
COUNTER = -1
PERWEEK_AGGREGATION = True
FILE_NAME = 'enron_graph_week.pickle' if PERWEEK_AGGREGATION else 'enron_graph_day.pickle'

def mail2nick(mail):
    global COUNTER
    if mail in MAIL2NICK:
        return MAIL2NICK[mail]
    
    COUNTER += 1
    MAIL2NICK[mail] = str(COUNTER)
    NICK2MAIL[str(COUNTER)] = mail
    
    return COUNTER

def nickn_streaming_graph(stream_graph):
    for date, graph in STREAMING_GRAPH.items():
        for sender, recivers_dict in graph.items():
            mail2nick(sender)
            for reciver, sender_revicer_mail in recivers_dict.items():
                mail2nick(reciver)
            

def jn(p1, p2):
    return osp.join(p1, p2)

def jnb(p):
    return jn(base_path, p)

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

def count_mails(graph):
    n = 0
    for sender, recivers_dict in graph.items():
        for reciver, sender_revicer_mail in recivers_dict.items():
            n += 1
    return n


%matplotlib inline

In [ ]:
class Mapping:
    
    def __init__(self):
        self.__dict__ = dict()

    def __setitem__(self, key, item):
        self.__dict__[key] = item

    def __getitem__(self, key):
        return self.__dict__[key]

    def __repr__(self):
        return repr(self.__dict__)

    def __len__(self):
        return len(self.__dict__)

    def __delitem__(self, key):
        del self.__dict__[key]

    def clear(self):
        return self.__dict__.clear()

    def copy(self):
        return self.__dict__.copy()

    def has_key(self, k):
        return k in self.__dict__

    def update(self, *args, **kwargs):
        return self.__dict__.update(*args, **kwargs)

    def keys(self):
        return self.__dict__.keys()

    def values(self):
        return self.__dict__.values()

    def items(self):
        return self.__dict__.items()

    def pop(self, *args):
        return self.__dict__.pop(*args)

    def __cmp__(self, dict_):
        return self.__cmp__(self.__dict__, dict_)

    def __contains__(self, item):
        return item in self.__dict__

    def __iter__(self):
        return iter(self.__dict__)

    def __unicode__(self):
        return unicode(repr(self.__dict__))

# Stream Graph implementation

In [ ]:
class Edge(Mapping):
    
    def __init__(self, d={}):
        super().__init__()
        self.__dict__ = d
        
    def __add__(self, other):
        cur = deepcopy(self)
        for key, val in other.items():
            if key in cur:
                cur[key] += deepcopy(val)
            else:
                cur[key] = deepcopy(val)
        return cur
    
    def __repr__(self):
        return str(self.__dict__)
    
def ddict():
    return defaultdict(Edge)
    
class Graph(Mapping):
    
    def __init__(self):
        #self.__dict__ = defaultdict(lambda : defaultdict(Edge))
        self.__dict__ = defaultdict(ddict)
    
    # srs - sourse mail addres
    # dst - destination mail addres
    # features - dict of key value features
    def add_pair(self, srs, dst, features):
        self[srs][dst] += Edge(features)
        
    
    def __repr__(self):
        return str(self.__dict__)
        

class StreamingGraph(Mapping):
    
    def __init__(self):
        self.__dict__ = defaultdict(Graph)
        
    # date - mail date
    # srs - sourse mail addres
    # dst - destination mail addres
    # features - dict of key value features
    def add_pair(self, date, srs, dst, features):
        self[date].add_pair(srs, dst, features)
        
    def __repr__(self):
        return str(self.__dict__)
    
    
def merge_streamin_graphs(lst_graphs):
    merged_graph = StreamingGraph()
    
    for stream_graph in tqdm.tqdm(lst_graphs):
        for day, day_graph in stream_graph.items():
            for srs, dst_dict in day_graph.items():
                for dst, dst_mail in dst_dict.items():
                    merged_graph.add_pair(day, srs, dst, dst_mail.__dict__)
                    
    res = {key: merged_graph[key] for key in sorted(merged_graph.keys())}
    return res

In [ ]:
def parce_mail(pth):
    mail = mailparser.parse_from_file(pth)
    
    d = {}
    dt = mail.Date[:-6]
    mask = '%a, %d %b %Y %H:%M:%S %z'
    try:
        d['Date'] = datetime.datetime.strptime(dt, mask)
    except:
        mask = '%a, %d %b %Y %H:%M:%S'
        try:
            d['Date'] = datetime.datetime.strptime(dt, mask)
        except Exception as e:
            print('ERROR')
            print(pth)
            print(e)
            print('----------------------------------------------------------------')
            return None
        
    
    d['From'] = [m.lower() for _, m in mail.From]
    d['To'] = [m.lower() for _, m in mail.To]
    d['CC'] = [m.lower() for _, m in mail.CC]
    d['BCC'] = [m.lower() for _, m in mail.BCC]
    d['Subject'] = mail.Subject.lower()
    #d['Body'] = re.sub(r'\s', ' ', mail.body.lower())
    return d


def parce_mail_dict(streaming_graph, mail_dict):
    if mail_dict is None:
        return
    
    edges_types = {'To': 2, 'CC': 3, 'BCC': 4}
    
    date = mail_dict['Date']
    week = week_of_month(date)
    # accumulate per week
    if PERWEEK_AGGREGATION:
        date_key = datetime.datetime(date.year, date.month, week)
    else:
        date_key = datetime.datetime(date.year, date.month, date.day)
        
    if date_key < MIN_DATE:
        return streaming_graph
    
    source = mail_dict['From'][0]
    subject = [mail_dict['Subject']]
    
    for dst_type in ['To', 'CC', 'BCC']:
        for dst in mail_dict[dst_type]:
            edge_features = {
                'edge_weight': 1,
                'edge_type': edges_types[dst_type],
                'Subject': subject,
                'isre': 'Re:' in subject
            }
            streaming_graph.add_pair(date_key, source, dst, edge_features)

    return streaming_graph


In [ ]:
def scan_mails(streaming_graph, pth):
    if streaming_graph is None:
        streaming_graph = StreamingGraph()
        
    if osp.isdir(pth):
        for sub_pth in listdir(pth):
            streaming_graph = scan_mails(streaming_graph, jn(pth, sub_pth))
        return streaming_graph
    else:
        streaming_graph = parce_mail_dict(streaming_graph, parce_mail(pth))
        return streaming_graph
        
def scan_task(pth):
    return scan_mails(None, pth[0])
    

def build_graph():
    graphs = []
    with Pool(processes=32) as p:
        pathes = [(jn(base_path, employ), i) for i, employ in enumerate(listdir(base_path))]
        with tqdm.tqdm(total=len(pathes)) as pbar:
            for i, stream_graph in enumerate(p.imap_unordered(scan_task, pathes)):
                graphs.append(stream_graph)
                pbar.update()
    return merge_streamin_graphs(graphs)

# Построение графа

In [ ]:
%%time
STREAMING_GRAPH = build_graph()

# Обход графа (не полный)

In [ ]:
for date, graph in STREAMING_GRAPH.items():
    print(f'DATE: {date}\n')
    for sender, recivers_dict in graph.items():
        print(f'    FROM: {sender}')
        for reciver, sender_revicer_mail in recivers_dict.items():
            print(f'        TO: {reciver}')
            for key, val in sender_revicer_mail.items():
                print(f'        {key}: {val}')
            break
        break
    print('\n\n___________________________________________________________________________________\n')
    break
    

# Pickle Save/Load

In [ ]:
with open(f'/mnt/data/molchanov/datasets/pickles/{FILE_NAME}', 'wb') as f:
    pickle.dump(STREAMING_GRAPH, f)

In [ ]:
with open(f'/mnt/data/molchanov/datasets/pickles/{FILE_NAME}', 'rb') as f:
    STREAMING_GRAPH = pickle.load(f)
    STREAMING_GRAPH = {k: v for k, v in STREAMING_GRAPH.items() if count_mails(v) > 140}
    nickn_streaming_graph(STREAMING_GRAPH)
    
for date, graph in STREAMING_GRAPH.items():
    print(f'DATE: {date}\n')
    for sender, recivers_dict in graph.items():
        print(f'    FROM: {sender}({MAIL2NICK[sender]})')
        for reciver, sender_revicer_mail in recivers_dict.items():
            print(f'        TO: {reciver}({MAIL2NICK[reciver]})')
            for key, val in sender_revicer_mail.items():
                print(f'        {key}: {val}')
            break
        break
    print('\n\n___________________________________________________________________________________\n')
    break

In [ ]:
def get_graph_nodes(graph):
    nodes = set()
    for srs, dst_dict_mails in graph.items():
        nodes.add(srs)
        for dst, _ in dst_dict_mails.items():
            nodes.add(dst)
    return nodes

def get_graph_edges(graph):
    edges = set()
    for srs, dst_dict_mails in graph.items():
        for dst, _ in dst_dict_mails.items():
            edges.add(f'{srs}|{dst}')
    return edges

def filter_graph(day_key, graph, blocked_nodes):
    filtered_graph = Graph()
    
    block = blocked_nodes[day_key] if isinstance(blocked_nodes, dict) else blocked_nodes
    
    for srs_mail, dict_dst_mails in graph.items():
        
        if srs_mail in block:
            continue
            
        d = {k: v for k, v in dict_dst_mails.items() if k not in block}
        if len(list(d.keys())) == 0:
            continue
            
        filtered_graph[srs_mail] = d

    if len(list(filtered_graph.keys())) == 0:
        return None
    return filtered_graph

def filter_stream_graph(stream_graph, blocked_nodes):
    g = StreamingGraph()
    tmp = {k: filter_graph(k, v, blocked_nodes) for k, v in stream_graph.items()}
    g.__dict__ = {k: v for k, v in tmp.items() if v is not None}
    return g    

# Some statistic

In [ ]:
'grpahs count ', len(list(STREAMING_GRAPH.keys())), 'nodes count ', len(list(MAIL2NICK.keys()))

### Ноды spam

In [ ]:
def get_spam_nodes(stream_graph):
    blocked_nodes = dict()#set()
    for date, graph in stream_graph.items():
        senders = np.array(list(graph.keys()))
        senders_activity = np.zeros(len(senders))

        for i, sender in enumerate(senders):
            for reciver, sender_revicer_mail in graph[sender].items():
                senders_activity[i] += 1

        max_activity = max(20, np.quantile(senders_activity, 0.95))
        mask = senders_activity >= max_activity
        #blocked_nodes = blocked_nodes.union(set(list(senders[mask])))
        blocked_nodes[date] = set(list(senders[mask]))
    return blocked_nodes


### Фильтрация графа от спам нод

In [ ]:
spam_nodes = get_spam_nodes(STREAMING_GRAPH)
STREAMING_GRAPH = filter_stream_graph(STREAMING_GRAPH, spam_nodes)
'spam_nodes count', len(spam_nodes) ,'grpahs count ', len(list(STREAMING_GRAPH.keys()))

### Активность нод во времени

In [ ]:
# без учета активности внутри дня (+1/+0 если была нода активна/неактивна в течении дня, 
# не учитываем число активностей ноды внутри 1 дня)

def activity_nodes_in_time(stream_graph):
    nodes_count = len(list(MAIL2NICK.keys()))
    nodes_activity = np.zeros(nodes_count)

    for date, graph in stream_graph.items():
        s_in = set() 
        for sender, recivers_dict in graph.items():
            if MAIL2NICK[sender] not in s_in:
                nodes_activity[int(MAIL2NICK[sender])] += 1
                s_in.add(MAIL2NICK[sender])

            for reciver, sender_revicer_mail in recivers_dict.items():
                if MAIL2NICK[reciver] not in s_in:
                    nodes_activity[int(MAIL2NICK[reciver])] += 1
                    s_in.add(MAIL2NICK[reciver])


    q = 0.95           
    print(f'{q} quantile activity: {np.quantile(nodes_activity, q)}')
    min_activity = np.quantile(nodes_activity, q)

    plt.rcParams.update({'font.size': 22})
    plt.figure(figsize=(40,10))
    plt.title(['Активность нод без учета активности внутри дня'])
    plt.scatter(np.arange(nodes_activity.shape[0]), np.sort(nodes_activity), color='red')
    plt.plot(np.arange(nodes_activity.shape[0]), [min_activity] * nodes_activity.shape[0], color='black')
    plt.show()
    plt.close()
    
    low_activiti_nodes = np.arange(nodes_count)
    mask = nodes_activity < min_activity
    low_activiti_nodes = low_activiti_nodes[mask]
    
    return set([NICK2MAIL[str(node)] for node in low_activiti_nodes])

low_activity_nodes = activity_nodes_in_time(STREAMING_GRAPH)
'low activity nodes count', len(low_activity_nodes)


In [ ]:
STREAMING_GRAPH = filter_stream_graph(STREAMING_GRAPH, low_activity_nodes)
'spam_nodes count', len(spam_nodes) ,'grpahs count ', len(list(STREAMING_GRAPH.keys()))

### IOU общих нод между двумя последовательными графами

In [ ]:
def get_graph_nodes(graph):
    nodes = set()
    for srs_mail, dict_dst_mails in graph.items():
        nodes.add(MAIL2NICK[srs_mail])
        for dst_mail, mail_info in dict_dst_mails.items():
            nodes.add(MAIL2NICK[dst_mail])
    return nodes

def successive_ious():
    ious = []
    for i in range(1, len(list(STREAMING_GRAPH.keys()))):
        day_key1, day_key2 = list(STREAMING_GRAPH.keys())[i - 1], list(STREAMING_GRAPH.keys())[i]
        graph1, graph2 = STREAMING_GRAPH[day_key1], STREAMING_GRAPH[day_key2]
        nodes1, nodes2 = get_graph_nodes(graph1), get_graph_nodes(graph2)

        intersec = len(nodes1.intersection(nodes2))
        union = len(nodes1.union(nodes2))

        iou = float(intersec)/union
        ious.append(iou)
    
    print(f'IOUS mean: {np.array(ious).mean()}, std: {np.array(ious).std()}')
    plt.rcParams.update({'font.size': 22})
    plt.figure(figsize=(40,10))
    plt.title(['IOU общих нод между двумя последовательными графами'])
    plt.plot(np.arange(len(ious)), ious, color='red')
    
successive_ious()


# Graph Visualisation

In [ ]:
def draw_day_graph(day):
    day_key = list(STREAMING_GRAPH.keys())[day]
    day_graph = STREAMING_GRAPH[day_key]
    
    edges = []
    for srs_mail, dict_dst_mails in day_graph.items():
        for dst_mail, mail_info in dict_dst_mails.items():
            edges.append((MAIL2NICK[srs_mail], MAIL2NICK[dst_mail]))
            
    mails_count = count_mails(day_graph)
            
    print(f'day {day_key} num edges = {len(edges)}')
    
    
    G=nx.Graph()
    G.add_edges_from(edges)
    #print(f'nodes: {G.nodes()}')
    
    nx.draw(G, with_labels=True)
    plt.figure(figsize=(80,80))
    #plt.savefig(f"{day_key.year}_{day_key.month}_{day_key.day}.png") # save as png
    plt.show() # display
    plt.close()
    
    
    
for i in range(len(list(STREAMING_GRAPH.keys()))):
    draw_day_graph(i)